# SecondPieceRandomForest


In [ ]:
import pandas as pd

data = pd.read_excel('df2.xlsx')

data.to_csv('df2.csv', index=False)


In [ ]:
#data = pd.read_csv('FullData.csv')
print(data.columns)
data2=data.copy()


In [ ]:
cat_cols = data.columns
for col in cat_cols:
    data[col] = data[col].apply(lambda x: x.lower() if isinstance(x, str) else x)


In [ ]:
from datetime import datetime, timedelta

data['time'] = data.apply(lambda row: (datetime.strptime(f"{row['TIMEHR']}:{row['TIMEMIN']} {row['AMPM']}", "%H:%M %p") + pd.Timedelta(hours=12)).strftime("%H:%M:%S") if row['AMPM'] == 'PM' else datetime.strptime(f"{row['TIMEHR']}:{row['TIMEMIN']} {row['AMPM']}", "%I:%M %p").strftime("%H:%M:%S"), axis=1)

data = data.drop(['TIMEHR', 'TIMEMIN', 'AMPM'], axis=1)


In [ ]:
data['time'] = data['time'].astype(str)
data['time'] = pd.to_datetime(data['time'], format='%H:%M:%S')

data['hour'] = data['time'].dt.hour
data['minute'] = data['time'].dt.minute

data = data.drop('time', axis=1)


In [ ]:
databeforeNA= data.copy()

num_rows = databeforeNA.shape[0]
print("# of records before cleaning", num_rows)
databeforeNA = databeforeNA.dropna()
num_rows = databeforeNA.shape[0]
print("# of records after NA removal", num_rows)


In [ ]:
datawithTime=databeforeNA.copy()


In [ ]:
unique_values0 = datawithTime['RAILROAD'].unique()
unique_values1 = datawithTime['STATE '].unique()
unique_values2 = datawithTime['CAUSE'].unique()
unique_values4 = datawithTime['TYPEQ'].unique()
unique_values5 = datawithTime['TRKCLAS'].unique()



In [ ]:
from sklearn.preprocessing import LabelEncoder
datawithTime = databeforeNA.copy()
le = LabelEncoder()
cat_cols = ['RAILROAD','STATE ', 'CAUSE']

for col in cat_cols:
    if datawithTime[col].dtype == 'object':
        datawithTime[col] = le.fit_transform(datawithTime[col])

typeq_dict = {'1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, 'a': 10, 'b': 11, 'c': 12, 'd': 13, 'e': 14}
datawithTime['TYPEQ'] = datawithTime['TYPEQ'].astype(str).map(typeq_dict)



In [ ]:

from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression

target_cols = ['ACCDMG', 'CASKLD', 'CASINJ', 'EVACUATE', 'CARSDMG', 'CARSHZD']
feature_cols = ['RAILROAD', 'YEAR','MONTH', 'DAY', 'CARS', 'STATE ', 'TEMP', 'VISIBLTY', 'WEATHER',
                'TRNSPD', 'TONS', 'TYPEQ', 'TRKCLAS', 'TYPTRK', 'POSITON1',
                'HEADEND1', 'LOADF1', 'EMPTYF1', 'CAUSE', 'ACCTRK',
                'HIGHSPD', 'hour', 'minute']
datawithTime.to_csv('datawithTime.csv', index=False)
data_FSelection = pd.read_csv('datawithTime.csv')

X = data_FSelection[feature_cols]

k = 5

for target_col in target_cols:
    y = data_FSelection[target_col]

    selector = SelectKBest(score_func=f_classif, k=k)
    selector.fit(X, y)

    top_k_indices = selector.get_support(indices=True)

    top_k_features = [feature_cols[i] for i in top_k_indices]
    print('Selected Features:', top_k_features)

    X_top_k = X[top_k_features]
    linreg = LinearRegression()
    scores = cross_val_score(linreg, X_top_k, y, cv=5)

    print(f'Accuracy score for {target_col}: {scores.mean()}')



In [ ]:
import numpy as np
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

target_cols = ['ACCDMG','YEAR', 'CASKLD', 'CASINJ', 'EVACUATE', 'CARSDMG', 'CARSHZD']
feature_cols = ['RAILROAD', 'YEAR', 'MONTH', 'DAY', 'CARS', 'STATE ', 'TEMP', 'VISIBLTY', 'WEATHER',
                'TRNSPD', 'TONS', 'TYPEQ', 'TRKCLAS', 'TYPTRK', 'POSITON1',
                'HEADEND1', 'LOADF1', 'EMPTYF1', 'CAUSE', 'ACCTRK',
                'HIGHSPD', 'hour', 'minute']

datawithTime.to_csv('datawithTime.csv', index=False)
data_FSelection = pd.read_csv('datawithTime.csv')

X = data_FSelection[feature_cols]
y = data_FSelection[target_cols]

k = 5

rfc = RandomForestClassifier(n_estimators=100, random_state=42)
selector = SelectFromModel(rfc, threshold=-np.inf, max_features=k)
selector.fit(X, y)

top_k_indices = selector.get_support(indices=True)

top_k_features = [feature_cols[i] for i in top_k_indices]
print('Selected Features:', top_k_features)

X_top_k = X[top_k_features]
rfc = RandomForestClassifier(n_estimators=100, random_state=42)
scores = cross_val_score(rfc, X_top_k, y, cv=5)

for i, target_col in enumerate(target_cols):
    print(f'Accuracy score for {target_col}: {scores[i]}')


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

data = pd.read_excel('df2.xlsx')

data.to_csv('df2.csv', index=False)

data2=data.copy()

cat_cols = data.columns
for col in cat_cols:
    data[col] = data[col].apply(lambda x: x.lower() if isinstance(x, str) else x)

from datetime import datetime, timedelta

data['time'] = data.apply(lambda row: (datetime.strptime(f"{row['TIMEHR']}:{row['TIMEMIN']} {row['AMPM']}", "%H:%M %p") + pd.Timedelta(hours=12)).strftime("%H:%M:%S") if row['AMPM'] == 'PM' else datetime.strptime(f"{row['TIMEHR']}:{row['TIMEMIN']} {row['AMPM']}", "%I:%M %p").strftime("%H:%M:%S"), axis=1)

data = data.drop(['TIMEHR', 'TIMEMIN', 'AMPM'], axis=1)

data['time'] = data['time'].astype(str)
data['time'] = pd.to_datetime(data['time'], format='%H:%M:%S')

data['hour'] = data['time'].dt.hour
data['minute'] = data['time'].dt.minute

data = data.drop('time', axis=1)

databeforeNA= data.copy()

num_rows = databeforeNA.shape[0]
print("# of records before cleaning", num_rows)
databeforeNA = databeforeNA.dropna()
num_rows = databeforeNA.shape[0]
print("# of records after NA removal", num_rows)

datawithTime=databeforeNA.copy()
print(datawithTime.head())

datawithTime['RAILROAD'] = datawithTime['RAILROAD'].str.lower()
datawithTime['STATE '] = datawithTime['STATE '].str.lower()
datawithTime['CAUSE'] = datawithTime['CAUSE'].str.lower()
datawithTime['TYPEQ'] = datawithTime['TYPEQ'].str.lower()

unique_values0 = datawithTime['RAILROAD'].unique()
unique_values1 = datawithTime['STATE '].unique()
unique_values2 = datawithTime['CAUSE'].unique()
unique_values4 = datawithTime['TYPEQ'].unique()
unique_values5 = datawithTime['TRKCLAS'].unique()

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(datawithTime['RAILROAD'])

data['RAILROAD encoded'] = le.transform(data['RAILROAD'])

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
cat_cols = ['RAILROAD','STATE ', 'CAUSE','TYPEQ']
for col in cat_cols:
    print(col)
    if datawithTime[col].dtype == 'object':
        datawithTime[col] = le.fit_transform(datawithTime[col])

from sklearn.preprocessing import LabelEncoder
datawithTime = databeforeNA.copy()
print(datawithTime['TYPEQ'].unique())
le = LabelEncoder()
cat_cols = ['RAILROAD','STATE ', 'CAUSE']

for col in cat_cols:
    if datawithTime[col].dtype == 'object':
        datawithTime[col] = le.fit_transform(datawithTime[col])

typeq_dict = {'1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, 'a': 10, 'b': 11, 'c': 12, 'd': 13, 'e': 14}
datawithTime['TYPEQ'] = datawithTime['TYPEQ'].astype(str).map(typeq_dict)

one_hot = pd.get_dummies(data['STATE '])
train_services_encoded = pd.get_dummies(data['RAILROAD'])

features = pd.concat([data.drop('STATE ', axis=1), one_hot,train_services_encoded], axis=1)

from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression

target_cols = ['ACCDMG', 'CASKLD', 'CASINJ', 'EVACUATE', 'CARSDMG', 'CARSHZD']
feature_cols = ['RAILROAD', 'YEAR','MONTH', 'DAY', 'CARS', 'STATE ', 'TEMP', 'VISIBLTY', 'WEATHER',
               'TRNSPD', 'TONS', 'TYPEQ', 'TRKCLAS', 'TYPTRK', 'POSITON1',
               'HEADEND1', 'LOADF1', 'EMPTYF1', 'CAUSE', 'ACCTRK',
               'HIGHSPD', 'hour', 'minute']

datawithTime.to_csv('datawithTime.csv', index=False)
data_FSelection = pd.read_csv('datawithTime.csv')

#import numpy as np
#import numpy as np
#import numpy as np

data = pd.read_csv('datawithTime.csv', nrows=int(1*len(pd.read_csv('datawithTime.csv'))))

target_cols = ['ACCDMG', 'CASKLD', 'CASINJ', 'EVACUATE', 'CARSDMG', 'CARSHZD']
all_feature_cols = ['RAILROAD', 'YEAR', 'MONTH', 'DAY', 'CARS', 'STATE ', 'TEMP', 'VISIBLTY', 'WEATHER',
                    'TRNSPD', 'TONS', 'TYPEQ', 'TRKCLAS', 'TYPTRK', 'POSITON1',
                    'HEADEND1', 'LOADF1', 'EMPTYF1', 'CAUSE', 'ACCTRK',
                    'HIGHSPD', 'hour', 'minute']

X = data[all_feature_cols]
y = data[target_cols]

for target in target_cols:
    y_single = y[target]

    k = 23
    selector = SelectKBest(f_classif, k=k)
    X_new = selector.fit_transform(X, y_single)

    selected_feature_indices = selector.get_support(indices=True)
    selected_feature_cols = [all_feature_cols[i] for i in selected_feature_indices]

    X_train, X_test, y_train, y_test = train_test_split(X_new, y_single, test_size=0.3, random_state=42)

    rf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
    rf.fit(X_train, y_train)

    X_test_new = selector.transform(X_test)
    y_pred = rf.predict(X_test_new)

    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy for", target, ":", accuracy)

    confusion = confusion_matrix(y_test, y_pred)
    print("Confusion matrix for", target, ":\n", confusion)

    print("Selected feature columns:", selected_feature_cols)


In [ ]:
"""ForComputeCanada.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1dBimbAjM6HqGHgqK2Py19lkQCkZVG_eO
"""

import pandas as pd

data = pd.read_excel('df2.xlsx')

data.to_csv('df2.csv', index=False)

print (data.dtypes)

#data = pd.read_csv('FullData.csv')
print(data.columns)
data2=data.copy()

cat_cols = data.columns
for col in cat_cols:
    data[col] = data[col].apply(lambda x: x.lower() if isinstance(x, str) else x)

print(data2)

from datetime import datetime, timedelta

data['time'] = data.apply(lambda row: (datetime.strptime(f"{row['TIMEHR']}:{row['TIMEMIN']} {row['AMPM']}", "%H:%M %p") + pd.Timedelta(hours=12)).strftime("%H:%M:%S") if row['AMPM'] == 'PM' else datetime.strptime(f"{row['TIMEHR']}:{row['TIMEMIN']} {row['AMPM']}", "%I:%M %p").strftime("%H:%M:%S"), axis=1)

data = data.drop(['TIMEHR', 'TIMEMIN', 'AMPM'], axis=1)

print(data)

print(data[['time']].head())
print(data2[['TIMEHR','TIMEMIN','AMPM']].head())

print(data.dtypes)

data['time'] = data['time'].astype(str)
data['time'] = pd.to_datetime(data['time'], format='%H:%M:%S')

data['hour'] = data['time'].dt.hour
data['minute'] = data['time'].dt.minute

data = data.drop('time', axis=1)

print(data)

print(data.columns)

databeforeNA= data.copy()

num_rows = databeforeNA.shape[0]
print("# of records before cleaning", num_rows)
databeforeNA = databeforeNA.dropna()
num_rows = databeforeNA.shape[0]
print("# of records after NA removal", num_rows)

datawithTime=databeforeNA.copy()
print(datawithTime.head())

unique_values0 = datawithTime['RAILROAD'].unique()
unique_values1 = datawithTime['STATE '].unique()
unique_values2 = datawithTime['CAUSE'].unique()
unique_values4 = datawithTime['TYPEQ'].unique()
unique_values5 = datawithTime['TRKCLAS'].unique()

print(len(unique_values0))
print(len(unique_values1))
print(len(unique_values2))
print(len(unique_values4))
print(len(unique_values5))

print(data.dtypes)

from sklearn.preprocessing import LabelEncoder
datawithTime = databeforeNA.copy()
print(datawithTime['TYPEQ'].unique())
le = LabelEncoder()
cat_cols = ['RAILROAD','STATE ', 'CAUSE']

for col in cat_cols:
    if datawithTime[col].dtype == 'object':
        datawithTime[col] = le.fit_transform(datawithTime[col])

typeq_dict = {'1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, 'a': 10, 'b': 11, 'c': 12, 'd': 13, 'e': 14}
datawithTime['TYPEQ'] = datawithTime['TYPEQ'].astype(str).map(typeq_dict)

print(datawithTime['TYPEQ'].unique())

print(datawithTime.dtypes)

print(datawithTime[['STATE ']].head())
print(datawithTime[['RAILROAD']].head())
print(datawithTime[['CAUSE']].head())
print(datawithTime[['TYPEQ']].head())

print(datawithTime.dtypes)

print(datawithTime.columns)

from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
data = pd.read_csv('datawithTime.csv', nrows=int(1*len(pd.read_csv('datawithTime.csv'))))

target_cols = ['ACCDMG', 'CASKLD', 'CASINJ', 'EVACUATE', 'CARSDMG', 'CARSHZD']
feature_cols = ['RAILROAD', 'YEAR', 'MONTH', 'DAY', 'CARS', 'STATE ', 'TEMP', 'VISIBLTY', 'WEATHER',
                'TRNSPD', 'TONS', 'TYPEQ', 'TRKCLAS', 'TYPTRK', 'POSITON1',
                'HEADEND1', 'LOADF1', 'EMPTYF1', 'CAUSE', 'ACCTRK',
                'HIGHSPD', 'hour', 'minute']

X = data[feature_cols]
y = data[target_cols]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

rf_models = {}
for target in target_cols:
    rf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
    rf.fit(X_train, y_train[target])
    rf_models[target] = rf

y_pred = {}
for target in target_cols:
    y_pred[target] = rf_models[target].predict(X_test)

from sklearn.metrics import confusion_matrix

confusion_matrices = {}
for target in target_cols:
    y_true = y_test[target]
    y_pred = y_pred[target]
    confusion_matrices[target] = confusion_matrix(y_true, y_pred)

for target, matrix in confusion_matrices.items():
    print("Confusion matrix for", target, ":")
    print(matrix)

accuracy = {}
for target in target_cols:
    accuracy[target] = accuracy_score(y_test[target], y_pred[target])
    print("Accuracy for", target, ":", accuracy[target])

target_importance_sum = {}

for target in target_cols:
    importances = rf_models[target].feature_importances_
    indices = np.argsort(importances)[::-1]
    print("\nFeature ranking for", target)
    target_importance_sum[target] = 0  # Initialize the summation to zero
    for f in range(X.shape[1]):
        feature_name = feature_cols[indices[f]]
        importance = importances[indices[f]]
        target_importance_sum[target] += importance  # Accumulate the importance
        print("%d. feature %s (%f)" % (f + 1, feature_name, importance))

print("\nSummation of importance for each target column:")
for target, importance_sum in target_importance_sum.items():
    print(target, ": ", importance_sum)



In [ ]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

data = pd.read_excel('df2.xlsx')

data.to_csv('df2.csv', index=False)

data2=data.copy()

cat_cols = data.columns
for col in cat_cols:
    data[col] = data[col].apply(lambda x: x.lower() if isinstance(x, str) else x)

from datetime import datetime, timedelta

data['time'] = data.apply(lambda row: (datetime.strptime(f"{row['TIMEHR']}:{row['TIMEMIN']} {row['AMPM']}", "%H:%M %p") + pd.Timedelta(hours=12)).strftime("%H:%M:%S") if row['AMPM'] == 'PM' else datetime.strptime(f"{row['TIMEHR']}:{row['TIMEMIN']} {row['AMPM']}", "%I:%M %p").strftime("%H:%M:%S"), axis=1)

data = data.drop(['TIMEHR', 'TIMEMIN', 'AMPM'], axis=1)

data['time'] = data['time'].astype(str)
data['time'] = pd.to_datetime(data['time'], format='%H:%M:%S')

data['hour'] = data['time'].dt.hour
data['minute'] = data['time'].dt.minute

data = data.drop('time', axis=1)

databeforeNA= data.copy()

num_rows = databeforeNA.shape[0]
print("# of records before cleaning", num_rows)
databeforeNA = databeforeNA.dropna()
num_rows = databeforeNA.shape[0]
print("# of records after NA removal", num_rows)

datawithTime=databeforeNA.copy()
print(datawithTime.head())

datawithTime['RAILROAD'] = datawithTime['RAILROAD'].str.lower()
datawithTime['STATE '] = datawithTime['STATE '].str.lower()
datawithTime['CAUSE'] = datawithTime['CAUSE'].str.lower()
datawithTime['TYPEQ'] = datawithTime['TYPEQ'].str.lower()

unique_values0 = datawithTime['RAILROAD'].unique()
unique_values1 = datawithTime['STATE '].unique()
unique_values2 = datawithTime['CAUSE'].unique()
unique_values4 = datawithTime['TYPEQ'].unique()
unique_values5 = datawithTime['TRKCLAS'].unique()

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(datawithTime['RAILROAD'])

data['RAILROAD encoded'] = le.transform(data['RAILROAD'])

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
cat_cols = ['RAILROAD','STATE ', 'CAUSE','TYPEQ']
for col in cat_cols:
    print(col)
    if datawithTime[col].dtype == 'object':
        datawithTime[col] = le.fit_transform(datawithTime[col])

from sklearn.preprocessing import LabelEncoder
datawithTime = databeforeNA.copy()
print(datawithTime['TYPEQ'].unique())
le = LabelEncoder()
cat_cols = ['RAILROAD','STATE ', 'CAUSE']

for col in cat_cols:
    if datawithTime[col].dtype == 'object':
        datawithTime[col] = le.fit_transform(datawithTime[col])

typeq_dict = {'1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, 'a': 10, 'b': 11, 'c': 12, 'd': 13, 'e': 14}
datawithTime['TYPEQ'] = datawithTime['TYPEQ'].astype(str).map(typeq_dict)

one_hot = pd.get_dummies(data['STATE '])
train_services_encoded = pd.get_dummies(data['RAILROAD'])

features = pd.concat([data.drop('STATE ', axis=1), one_hot,train_services_encoded], axis=1)

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import xgboost as xgb

data = pd.read_csv('datawithTime.csv')

target_cols = ['ACCDMG', 'CASKLD', 'CASINJ', 'EVACUATE', 'CARSDMG', 'CARSHZD']
feature_cols = ['RAILROAD', 'YEAR', 'MONTH', 'DAY', 'CARS', 'STATE ', 'TEMP', 'VISIBLTY', 'WEATHER',
                'TRNSPD', 'TONS', 'TYPEQ', 'TRKCLAS', 'TYPTRK', 'POSITON1',
                'HEADEND1', 'LOADF1', 'EMPTYF1', 'CAUSE', 'ACCTRK',
                'HIGHSPD', 'hour', 'minute']

X = data[feature_cols]
y = data[target_cols]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

xgb_model = xgb.XGBClassifier(objective='multi:softmax', num_class=len(target_cols), random_state=42)

# Train the model on the training data
xgb_model.fit(X_train, y_train)

y_pred = xgb_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
